In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import gc
from tqdm import tqdm

%matplotlib inline

PATH = '/kaggle/input/idao2020-track1/'

In [2]:
!pip install /kaggle/input/lib-plyades

Processing /kaggle/input/lib-plyades
  Created wheel for Plyades: filename=Plyades-0.0.1-cp36-none-any.whl size=18914 sha256=4d4738e5797b994c57e1c10721ef026123d3e05f8ddbbfca871621fbed9cdcf1
  Stored in directory: /root/.cache/pip/wheels/91/67/f4/51685f7b31e174b1573e464db1cc864edffe5d36c233556335
Successfully built Plyades


In [3]:
!pip install jplephem

  Created wheel for jplephem: filename=jplephem-2.12-cp36-none-any.whl size=43929 sha256=03847a99f665577a82ef2bf03fb46da8429201a6b1ce26e033f47692ce7ae063
  Stored in directory: /root/.cache/pip/wheels/08/ce/59/bde14a0c20cdfb538a28d78fec79dd628c8d03d5a75f812d1a
Successfully built jplephem


In [4]:
import plyades as pl
from astropy.time import Time
import astropy.units as units

In [5]:
%%time
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'test.csv')
submission = pd.read_csv(PATH + 'submission.csv')

CPU times: user 3.77 s, sys: 515 ms, total: 4.28 s
Wall time: 4.39 s


In [6]:
test.head()

,id,sat_id,epoch,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,3927,1,2014-02-01T00:01:45.162,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774
1,3928,1,2014-02-01T00:22:57.007,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402
2,3929,1,2014-02-01T00:44:08.852,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412
3,3930,1,2014-02-01T01:05:20.697,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600
4,3931,1,2014-02-01T01:26:32.542,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724


In [7]:
def make_iss(x, y, z, Vx, Vy, Vz, time):
    iss_r = np.array([x, y, z]) * units.km
    iss_v = np.array([Vx, Vy, Vz]) * units.km/units.s
    iss_t = Time(time)
    frame = 'ECI'
    body = pl.bodies.EARTH
    iss = pl.State(iss_r, iss_v, iss_t, frame, body)
    return iss

def generate_all_features(x, y, z, Vx, Vy, Vz, time):
    iss = make_iss(x, y, z, Vx, Vy, Vz, time)
    return (
        iss.semi_major_axis.value,
        iss.eccentricity.value,
        iss.inclination.value,
        iss.ascending_node.value,
        iss.argument_of_periapsis.value,
        iss.true_anomaly.value,
        iss.period.value,
        iss.orbital_energy.value,
        iss.mean_motion.value
    )

In [8]:
tqdm.pandas()
test[[
    'semi_major_axis',
    'eccentricity',
    'inclination',
    'ascending_node',
    'argument_of_periapsis',
    'true_anomaly',
    'period',
    'orbital_energy',
    'mean_motion'
]] = test.progress_apply(lambda s: pd.Series(generate_all_features(s['x_sim'], s['y_sim'], s['z_sim'], s['Vx_sim'], s['Vy_sim'], s['Vz_sim'], s['epoch'])), axis=1)

 93%|█████████▎| 263061/284071 [2:26:20<10:49, 32.33it/s]

In [9]:
test.to_csv('test_plyades.csv', index=False)